In [2]:
# =====================================================
# 1. Install & Import Dependencies
# =====================================================
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

from xgboost import XGBClassifier

# Fairlearn - for disparity metrics
from fairlearn.metrics import MetricFrame
from fairlearn.metrics import false_positive_rate, true_positive_rate
from fairlearn.metrics import selection_rate
from fairlearn.datasets import fetch_diabetes_hospital
from sklearn.model_selection import train_test_split




In [4]:
# =====================================================
# 2. Synthetic Data Setup (Replace with your own data)
# =====================================================


data = fetch_diabetes_hospital()
X_raw = data.data    # This is a DataFrame
Y = data.target      # This is a pandas Series of labels


df = X_raw.copy()
df["target"] = Y

# 3. Downsample to ~8k points, stratifying by class (Y).
df_sampled = df.sample(
    n=8000, 
    random_state=42,
    stratify=df["target"]  
)

# 4. Separate features, labels, and sensitive attribute
Y_downsampled = df_sampled["target"]
A_downsampled = df_sampled["race"]          # Protected attribute
X_downsampled = df_sampled.drop(columns=["race", "target"])

# (Optional) Check the shape
print("X_downsampled shape:", X_downsampled.shape)
print("Y_downsampled shape:", Y_downsampled.shape)
print("A_downsampled shape:", A_downsampled.shape)

TypeError: NDFrame.sample() got an unexpected keyword argument 'stratify'

In [3]:
Y

NameError: name 'Y' is not defined

In [5]:
data = fetch_diabetes_hospital()

In [10]:
X_raw = data.data
Y = data.target

In [15]:
X_raw.head()

,race,gender,age,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,...,insulin,change,diabetesMed,medicare,medicaid,had_emergency,had_inpatient_days,had_outpatient_days,readmitted,readmit_binary
0,Caucasian,Female,'30 years or younger',Other,Referral,1,Other,41,0,1,...,No,No,No,False,False,False,False,False,NO,0
1,Caucasian,Female,'30 years or younger','Discharged to Home',Emergency,3,Missing,59,0,18,...,Up,Ch,Yes,False,False,False,False,False,>30,1
2,AfricanAmerican,Female,'30 years or younger','Discharged to Home',Emergency,2,Missing,11,5,13,...,No,No,Yes,False,False,False,True,True,NO,0
3,Caucasian,Male,'30-60 years','Discharged to Home',Emergency,2,Missing,44,1,16,...,Up,Ch,Yes,False,False,False,False,False,NO,0
4,Caucasian,Male,'30-60 years','Discharged to Home',Emergency,1,Missing,51,0,8,...,Steady,Ch,Yes,False,False,False,False,False,NO,0
